# Web Scraping Information of "playstation 4 slim" in eBay

In [2]:
import pymysql
import requests
from bs4 import BeautifulSoup
import re
from time import sleep
import os
import json
import pprint
import warnings
import prettytable

In [4]:
# To only include but-it-now items, add LH_BIN=1 in url.
# GET request's variable name for buy-it-now is LH_BIN.
# To modify search query including 100 items per search result page, add _ipg=100 in url.
# GET request's variable name corresponding to items per page searches is _ipg.

## Identify all the sponsored items

For the first 10 pages of 100 items/page, save all the URLs of sponsored items' pages to the file "sponsored.txt"  and all the URLs of non-sponsored items' pages to the file "non-sponsored.txt" in the same directory as the code.

In [ ]:
sponsored_num = 0
non_sponsored_num = 0
sponsored_list = []
non_sponsored_list = []
string = ''

# Using for loop to get the first 10 pages
for page_num in range(1,11):
    url = 'https://www.ebay.com/sch/i.html?_nkw=playstation+4+slim&LH_BIN=1&_ipg=100&_pgn=' + str(page_num)
    page = requests.get(url)
    page_content = BeautifulSoup(page.content, 'html.parser')
    
    # Fetch items on each page
    items = page_content.find_all('li', {'class':'s-item   '} and {'id':re.compile(r'^srp-river-results-listing[0-9]+$')})
    
    for i in range(len(items)):
        # Sponsored item has a class of tag <span> which has format like 's-'+'6 digits'
        target_tag = items[i].find_all('span', re.compile(r'^s-[a-z0-9]{6}$'))
        
        # Fetch the "SPONSORED" tag and check if the item is sponsored or not
        for j in range(len(target_tag)):
            string = string + target_tag[j].get_text()
        if string == 'SPONSORED':
            sponsored_url = items[i].find('a').get('href')
            #heading = items[i].find('h3').get_text()
            sponsored_list.append(sponsored_url)
            #print(heading + '\n' + sponsored_url + '\n')
            sponsored_num = sponsored_num + 1
        else:
            non_sponsored_url = items[i].find('a').get('href')
            non_sponsored_list.append(non_sponsored_url)
            non_sponsored_num = non_sponsored_num + 1
        # Set string to null for next iteration
        string = ''
    # Set a 10 seconds pause
    sleep(10)
    
print('sponsored_num:', sponsored_num)
print('non_sponsored_num:', non_sponsored_num)

In [6]:
# Write sponsored url results into txt file
with open('sponsored_url.txt', 'w') as f:
    for url in range(len(sponsored_list)):
        f.write(sponsored_list[url]+'\n')
f.close

<function TextIOWrapper.close()>

In [7]:
# Write non_sponsored url results into txt file
with open('non_sponsored_url.txt', 'w') as f2:
    for url in range(len(non_sponsored_list)):
        f2.write(non_sponsored_list[url]+'\n')
f2.close

<function TextIOWrapper.close()>

## Create two folders in the same directory to save "sponsored" and "non-sponsored" pages

In [ ]:
# Create a function for creating new folder
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        
# Creates folders in the current directory called sponsored and non-sponsored respectively
createFolder('./sponsored/')
createFolder('./non-sponsored/')

In [12]:
# Read all sponsored url into a list:
with open('sponsored_url.txt', 'r') as f:
    sponsored_list = f.readlines()
f.close

In [50]:
# Enter sponsored directory
os.chdir(os.getcwd()+'/sponsored')

for item in range(len(sponsored_list)):    
    try:
        # Connect to the url page
        page = requests.get(sponsored_list[item])
        page_content = BeautifulSoup(page.content, 'html.parser')
        # Get item-id from the url 
        item_id = re.findall('(\/)([0-9]+)(\?)', sponsored_list[item])[0][1]
        file_name = str(item_id) + '.htm'
        # Save page
        with open(file_name, 'w') as f:
            f.write(str(page_content))
            f.close
        # Set a 2 seconds pause
        sleep(2)
        
    except:
        print('Failed:', item)

In [4]:
os.chdir(os.getcwd()[:-10])
# Read all sponsored url into a list:
with open('non_sponsored_url.txt', 'r') as f:
    non_sponsored_list = f.readlines()
f.close

<function TextIOWrapper.close()>

In [61]:
# Enter non-sponsored directory
os.chdir(os.getcwd()+'/non-sponsored')

for item in range(len(non_sponsored_list)):    
    try:
        # Connect to the url page
        page = requests.get(non_sponsored_list[item])
        page_content = BeautifulSoup(page.content, 'html.parser')
        # Get item-id from the url 
        item_id = re.findall('(\/)([0-9]+)(\?)', non_sponsored_list[item])[0][1]
        file_name = str(item_id) + '.htm'
        # Save page
        with open(file_name, 'w') as f:
            f.write(str(page_content))
            f.close
        # Set a 2 seconds pause
        sleep(2)
        
    except:
        print('Failed:', item)

In [37]:
# Since there are 994 files saved in non_sponsored directory, it may conclude repeated item ids
# Save all the item ids into a list
item_ids = []
for item in non_sponsored_list:
    item_id = re.findall('(\/)([0-9]+)(\?)', item)[0][1]
    item_ids.append(item_id)

# Find repeated item ids
from collections import Counter
count = dict(Counter(item_ids))
repeat = {key : value for key , value in count.items() if value > 1}
print(repeat)
# There are 6 items repeated 2 times in the non-sponsored result.
# That's why I only have 994 files saved in non_sponsored directory.

{'293455479189': 2, '133319698055': 2, '333292683394': 2, '174154920955': 2, '372908034705': 2, '202895335954': 2}


In [44]:
# Save duplicated id into a list
repeat_item_id = list(repeat.keys())
repeat_item_id

# Define a function returning index of duplicates items in a list
def duplicates(lst, item): 
    return [i for i, x in enumerate(lst) if x == item]

# Find the 6 pair index of duplicate item
repeat_index = []
for id in repeat_item_id:
    repeat_index.append(duplicates(item_ids, id))
    
# Check whether the duplicate item ids are the same one with the same url
n = 1
for index in repeat_index:
    print("Duplicate id " + str(n) +":")
    print(non_sponsored_list[index[0]])
    print(non_sponsored_list[index[1]])
    n = n + 1
# Their urls are the same, so I keep 994 files in non-sponsored directory.

Duplicate id 1:
https://www.ebay.com/itm/Sony-CUH-2115A-Playstation-4-Slim-500GB-Game-Console-System-PS4/293455479189?hash=item44534f4195%3Ag%3AaPIAAOSw09ZeNc3V&LH_BIN=1

https://www.ebay.com/itm/Sony-CUH-2115A-Playstation-4-Slim-500GB-Game-Console-System-PS4/293455479189?hash=item44534f4195%3Ag%3AaPIAAOSw09ZeNc3V&LH_BIN=1

Duplicate id 2:
https://www.ebay.com/itm/Sony-Playstation-4-PS4-Slim-1TB-Console-Black-TESTED-CUH-2115B-w-7-Games/133319698055?epid=18029957672&hash=item1f0a792687%3Ag%3A-HMAAOSw0K5eEQeK%3Asc%3AUSPSPriority%2194122%21US%21-1&LH_BIN=1

https://www.ebay.com/itm/Sony-Playstation-4-PS4-Slim-1TB-Console-Black-TESTED-CUH-2115B-w-7-Games/133319698055?epid=18029957672&hash=item1f0a792687%3Ag%3A-HMAAOSw0K5eEQeK%3Asc%3AUSPSPriority%2194122%21US%21-1&LH_BIN=1

Duplicate id 3:
https://www.ebay.com/itm/PlayStation-4-1TB-Slim-NBA-2K18-Bundle-Edition-Console-Basketball-Black-Video/333292683394?epid=2280030462&hash=item4d99cac082%3Ag%3AYsAAAOSw7RNdTdOn&LH_BIN=1

https://www.ebay.co

## Identify and select elements
- seller name
- seller score
- item price
- items sold
- best offer available
- title
- returns allowed
- shipping price
- condition

In [139]:
# Check the working directory
os.getcwd()

'/Users/mokki/Desktop/422 - Data Design & Representation/MID/non-sponsored'

In [140]:
# For non-sponsored items:
item_data = []
item_info = []
counter = 0

for file_name in os.listdir(os.getcwd()):
    
    # I found a page(312973855034.htm) having totally different structure with other pages. 
    # Ebay shows "The listing you're looking for has ended."
    # It seems the page ended after I got the url but before I save the page.
    # So I excluded this page.
    
    if file_name.endswith(".htm") and file_name != '312973855034.htm': 
        
        page = open(file_name).read()
        soup = BeautifulSoup(page,'html.parser')
        
        # item id
        item_id = file_name.replace(".htm", "")
        item_info.append(item_id)
        
        # seller name
        seller_name = soup.find("span","mbg-nw").string
        item_info.append(seller_name)

        # seller score
        seller_score = soup.find("span","mbg-l").find("a").string
        item_info.append(seller_score)

        # item price
        if soup.find("span", id = "prcIsum") == None:
            if soup.find("span", itemprop = "price") == None:
                price = soup.find("span", id = "orgPrc").string
            else:
                price = soup.find("span", itemprop = "price").string
        else:
            price = soup.find("span", id = "prcIsum").string
            
        item_info.append(price)

        # items sold
        if soup.find('a', text=re.compile('sold')) != None:
            sold = soup.find('a', text=re.compile('sold')).string
        else:
            sold = None
            
        item_info.append(sold)

        # best offer available
        if soup.find("a", id = "boBtn_btn") == None:
            best_offer = "No Best Offer"
        else:
            best_offer = soup.find("a", id = "boBtn_btn").string.strip()
            
        item_info.append(best_offer)

        # title
        title = soup.find("h1").text[16:].strip()
        item_info.append(title)

        # returns allowed
        returns = soup.find("span", id = "vi-ret-accrd-txt").text.strip()
        item_info.append(returns)

        # shipping price
        if soup.find("span", id = "shSummary") != None:
            if soup.find("strong", "sh_gr_bld") != None:
                shipping = soup.find("strong", "sh_gr_bld").string.strip()
            elif soup.find("span", id = "shSummary").find("span", id="fshippingCost") != None:
                shipping = soup.find("span", id = "shSummary").find("span", id="fshippingCost").find("span").string.strip()
            elif soup.find("span", id = "shSummary").find("span", id="fShippingSvc") != None:
                if soup.find("span", id = "shSummary").find("span", id="fShippingSvc").find("span") != None:
                    shipping = soup.find("span", id = "shSummary").find("span", id="fShippingSvc").find("span").string.strip()
                elif soup.find("span", id = "shSummary").find("span", id="fShippingSvc") != None:
                    shipping = soup.find("span", id = "shSummary").find("span", id="fShippingSvc").string.strip()           
            else:
                shipping = soup.find("span", id = "shSummary").find("a").string.strip()       
        else:
            shipping = None
        
        item_info.append(shipping)

        # condition
        condition = soup.find("div", id = "vi-itm-cond").string
        item_info.append(condition)

        
        item_data.append(item_info)
        item_info = []
        counter = counter + 1

print('The total non-sponsored items are:', counter)

The total non-sponsored items are: 993


In [ ]:
# Check the non-sponsored dataset:
#item_data

In [142]:
# Change the working directory
os.chdir(os.getcwd()[:-14]+'/sponsored')
os.getcwd()

'/Users/mokki/Desktop/422 - Data Design & Representation/MID/sponsored'

In [143]:
# For sponsored items:
item_data_sponsored = []
item_info = []
counter = 0

for file_name in os.listdir(os.getcwd()):
    if file_name.endswith(".htm"): 
        page = open(file_name).read()
        soup = BeautifulSoup(page,'html.parser')
        
        # item id
        item_id = file_name.replace(".htm", "")
        item_info.append(item_id)
        
        # seller name
        seller_name = soup.find("span","mbg-nw").string
        item_info.append(seller_name)

        # seller score
        seller_score = soup.find("span","mbg-l").find("a").string
        item_info.append(seller_score)

        # item price
        if soup.find("span", id = "prcIsum") == None:
            if soup.find("span", itemprop = "price") == None:
                price = soup.find("span", id = "orgPrc").string
            else:
                price = soup.find("span", itemprop = "price").string
        else:
            price = soup.find("span", id = "prcIsum").string
            
        item_info.append(price)

        # items sold
        if soup.find('a', text=re.compile('sold')) != None:
            sold = soup.find('a', text=re.compile('sold')).string
        else:
            sold = None
            
        item_info.append(sold)

        # best offer available
        if soup.find("a", id = "boBtn_btn") == None:
            best_offer = "No Best Offer"
        else:
            best_offer = soup.find("a", id = "boBtn_btn").string.strip()
            
        item_info.append(best_offer)

        # title
        title = soup.find("h1").text[16:].strip()
        item_info.append(title)

        # returns allowed
        returns = soup.find("span", id = "vi-ret-accrd-txt").string.strip()
        item_info.append(returns)

        # shipping price
        if soup.find("span", id = "shSummary") != None:
            if soup.find("strong", "sh_gr_bld") != None:
                shipping = soup.find("strong", "sh_gr_bld").text.strip()
            elif soup.find("span", id = "shSummary").find("span", id="fshippingCost") != None:
                shipping = soup.find("span", id = "shSummary").find("span", id="fshippingCost").find("span").text.strip()
            elif soup.find("span", id = "shSummary").find("span", id="fShippingSvc") != None:
                if soup.find("span", id = "shSummary").find("span", id="fShippingSvc").find("span") != None:
                    shipping = soup.find("span", id = "shSummary").find("span", id="fShippingSvc").find("span").text.strip()
                elif soup.find("span", id = "shSummary").find("span", id="fShippingSvc") != None:
                    shipping = soup.find("span", id = "shSummary").find("span", id="fShippingSvc").text.strip()           
            else:
                shipping = soup.find("span", id = "shSummary").find("a").text.strip()       
        else:
            shipping = None
        
        item_info.append(shipping)

        # condition
        condition = soup.find("div", id = "vi-itm-cond").string
        item_info.append(condition)

        
        item_data_sponsored.append(item_info)
        item_info = []
        counter = counter + 1

print('The total sponsored items are:', counter)

The total sponsored items are: 120


In [ ]:
# Check the sponsored dataset:
#item_data_sponsored

## Clean the data and save it to MySql database

In [147]:
# Add sponsored(1) or non-sponsored(0) flag to each dataset and combine to list ps4
ps4 = []
for item in item_data:
    item.append("0")
    ps4.append(item)
    
for item in item_data_sponsored:
    item.append("1")
    ps4.append(item)

In [149]:
# Clean the data before insert into MySql
counter = 0
for item in ps4:
    
    # Convert price into "dollar-cent" format:
    item.append(item[3].split()[0])
    item[3] = re.sub('\D', '', item[3])
    
    # Convert item sold to number:
    if item[4] != None:
        item[4] = re.sub('\D', '', item[4])
    
    # Convert Best Offer into boolean:
    if item[5] == 'Make Offer':
        item[5] = "1"
    else:
        item[5] = "0"
    
    # Clean returns content:
    if item[7] == 'Seller does not accept returns':
        item[7] = "0"
    else:
        item[7] = "1"
    
    # Convert shipping price and create a new column to identify whether the shipping price is calculated:
    if item[8] != None:
        if "free" in item[8].lower():
            item[8] = "0"
            item.append("0")
        elif "$" in item[8]:
            item[8] = re.sub('\D', '', item[8])
            item.append("0")
        # For "calculate" and "Local pick-up offered.", insert NULL in shipping price column and set shipping_calculated = 1
        else:
            item[8] = None
            item.append("1")
    else:
        item.append("0")
    
    counter = counter + 1
print("The total rows are:", counter)

The total rows are: 1113


In [150]:
# There is one item without currency unit, change it to US
ps4[131][11] = 'US'

In [134]:
# Ignore warnings
warnings.filterwarnings("ignore")

# Database name is eBay
SQL_DB = "eBay"

# Define a function to create SQL database and table
def create_sql_table(SQL_TABLE, SQL_TABLE_DEF):
    try:
        
        # Connect to server:
        conn = pymysql.connect(host='localhost', user = 'root', password = '')
        cursor = conn.cursor()
        
        # SQL query to create database:
        query = "CREATE DATABASE IF NOT EXISTS " + SQL_DB + ";"
        cursor.execute(query)
        
        # SQL query to create table:
        query = "CREATE TABLE IF NOT EXISTS " + SQL_DB + "." + SQL_TABLE + " " + SQL_TABLE_DEF + ";"
        cursor.execute(query)
        
        # Close cursor and connect:
        cursor.close()
        conn.close()
        return

    except IOError as e:
        print(e)


In [135]:
# Define table
SQL_TABLE = "eBay_items"
SQL_TABLE_DEF = "(" + \
            "item_id CHAR(12) NOT NULL PRIMARY KEY" + \
            ", seller_name VARCHAR(50)" + \
            ", seller_score INT(10)" + \
            ", price INT(10)" + \
            ", sold INT(10)" + \
            ", best_offer INT(1)" + \
            ", title VARCHAR(200)" + \
            ", returns INT(1)" + \
            ", shipping_price INT(10)" + \
            ", item_condition VARCHAR(50)" + \
            ", sponsored INT(1)" + \
            ", currency VARCHAR(5)" + \
            ", shipping_calculated INT(1)" + \
            ")"

# Call function to create in MySql
create_sql_table(SQL_TABLE, SQL_TABLE_DEF)

In [152]:
# Write SQL insert query:
insert_query = "INSERT INTO " + SQL_TABLE + \
"(item_id, seller_name, seller_score, price, sold, best_offer, title, returns, shipping_price, item_condition, sponsored, currency, shipping_calculated)" + \
" VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"

# Insert the values list into MySql table:
try:
    conn = pymysql.connect(host='localhost', user = 'root', password = '', database = SQL_DB)
    cursor = conn.cursor()
    
    cursor.executemany(insert_query, ps4)
    conn.commit()
    cursor.close()
    conn.close()
    
except IOError as e:
    print(e)

## Using SQL to check the statistical summary

In [168]:
# For numerical/binary categorical column:
columns = ['seller_score', 'price', 'sold', 'best_offer', 'returns', 'shipping_price', 'shipping_calculated']

for column in columns:
    query = 'SELECT sponsored, item_condition, avg('+ column +') as average, max('+ column +') as max, min('+ column +') as min, round(stddev('+ column +'),2) as sd' + \
    ' FROM eBay_items GROUP BY sponsored, item_condition ORDER BY sponsored, item_condition;'
    
    try:
        conn = pymysql.connect(host='localhost', user = 'root', password = '', database = SQL_DB)
        cursor = conn.cursor()

        cursor.execute(query)
        
        # Use prettytable to pretty print the sql result
        titles = [title[0] for title in cursor.description]
        table = prettytable.PrettyTable(titles)
        for row in cursor.fetchall():
            table.add_row(row)
        print('============ ' + column + ' ============')
        print(table)
        print('\n')
        
        cursor.close()
        conn.close()
    
    except IOError as e:
        print(e)

============ seller_score ============
+-----------+--------------------------+-------------+---------+-----+-----------+
| sponsored |      item_condition      |   average   |   max   | min |     sd    |
+-----------+--------------------------+-------------+---------+-----+-----------+
|     0     | Manufacturer refurbished | 452388.6000 | 2261921 |  0  |  904766.2 |
|     0     |           New            |  10695.2962 |  255808 |  -1 |  32749.21 |
|     0     |         Open box         |  23327.5652 |  638097 |  0  |  103185.6 |
|     0     |    Seller refurbished    | 155885.1143 | 2261921 |  0  | 524335.59 |
|     0     |           Used           |  46496.8895 |  258023 |  0  |  73095.79 |
|     1     | Manufacturer refurbished |   196.3333  |   530   |  3  |   236.93  |
|     1     |           New            |  44457.4694 |  916460 |  0  | 155439.56 |
|     1     |         Open box         |   16.6667   |    35   |  5  |   13.12   |
|     1     |    Seller refurbished    |  13919.

In [169]:
# For non-numerical/binary categorical column:
columns = ['item_id', 'seller_name', 'title', 'item_condition', 'currency']

for column in columns:
    query = 'SELECT sponsored, item_condition, count(' + column + ') FROM eBay_items GROUP BY sponsored, item_condition ORDER BY sponsored, item_condition;'

    try:
        conn = pymysql.connect(host='localhost', user = 'root', password = '', database = SQL_DB)
        cursor = conn.cursor()

        cursor.execute(query)
        
        # Use prettytable to pretty print the sql result
        titles = [title[0] for title in cursor.description]
        table = prettytable.PrettyTable(titles)
        for row in cursor.fetchall():
            table.add_row(row)
        print('============ ' + column + ' ============')
        print(table)
        print('\n')
        
        cursor.close()
        conn.close()
    
    except IOError as e:
        print(e)


============ item_id ============
+-----------+--------------------------+----------------+
| sponsored |      item_condition      | count(item_id) |
+-----------+--------------------------+----------------+
|     0     | Manufacturer refurbished |       5        |
|     0     |           New            |      368       |
|     0     |         Open box         |       69       |
|     0     |    Seller refurbished    |       35       |
|     0     |           Used           |      516       |
|     1     | Manufacturer refurbished |       3        |
|     1     |           New            |       49       |
|     1     |         Open box         |       3        |
|     1     |    Seller refurbished    |       3        |
|     1     |           Used           |       62       |
+-----------+--------------------------+----------------+


============ seller_name ============
+-----------+--------------------------+--------------------+
| sponsored |      item_condition      | count(selle

## Brief summary of the difference between "sponsored" and "non-sponsored" items

- There are 993 rows for non-sponsored and 120 rows for sponsored items. 

- In each group, items in "Used" condition is the most and then "New".

- Seller score varies significantly between sponsored and non-sponsored items. Non-sponosored group has much higher seller score. But for other features like price, sold..., It's difficult to judge the difference between the two groups by just using simple statistics.